In [1]:
import numpy as np
import seaborn as sns
import pandas as pd

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
from sklearn.linear_model import LogisticRegression

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
#Cargo el archivo parquet
df = pd.read_parquet(r'dataset\diabetes_ok.parquet')

In [3]:
df.head(3)

,Age,BMI,Chol,TG,HDL,LDL,BUN,Diabetes
0,50,24,4.2,0.9,2.4,1.4,4.7,0
1,26,23,3.7,1.4,1.1,2.1,4.5,0
2,33,21,4.9,1.0,0.8,2.0,7.1,0


In [4]:
df.shape

(5131, 8)

In [5]:
# Selecciono las columnas para entrenar 'X' y la columna objetivo 'Y'
# Importante transformar a arrays para no tener qu eescalar a la hora de usar el modelo
array = df.values

X = array[:,0:7]
y = array[:,7]

### Division de datos de entrenamiento y prueba


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [8]:
print(X_train[:11])

[[55.       25.        3.52      1.96      1.22      1.67      3.8     ]
 [38.       23.        3.73      1.18      1.05      1.94      3.36    ]
 [50.       23.        4.99      2.28      1.15      3.01      3.56    ]
 [51.       22.        6.4       1.84      1.23      3.88      5.57    ]
 [58.       22.        4.94      2.96      1.1       2.66      4.65    ]
 [30.       31.        4.82      1.43      4.860753  4.860753  4.36    ]
 [61.       32.        5.63      1.49      1.85      3.04      3.74    ]
 [74.       22.        5.01      2.01      1.42      2.68      4.41    ]
 [36.       19.        4.59      1.03      1.63      2.55      5.39    ]
 [58.       27.        4.39      1.2       1.32      2.34      3.25    ]
 [44.       23.        5.25      0.96      1.68      2.72      3.26    ]]


### Escalado de funciones

In [10]:
sc = StandardScaler()# crea el objeto de escalado 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
print(X_train[:4])

[[ 0.44714378  0.10294408 -1.33902568  0.18870124 -0.35294672 -1.31152944
  -0.65889253]
 [-0.77207832 -0.36807712 -1.13077276 -0.4203156  -0.51906018 -1.02540068
  -0.92550027]
 [ 0.08854904 -0.36807712  0.11874479  0.4385543  -0.42134638  0.10851704
  -0.80431493]
 [ 0.16026799 -0.60358772  1.51701442  0.09500634 -0.34317534  1.03048752
   0.41359773]]


### Entrenamiento modelo regresion logistica

In [43]:
best_params = {'penalty': 'l2', 'C': 0.01} #Utilizo los hiperparametros resultado del analisis de hiperparametros.
model = LogisticRegression(random_state = 0, **best_params, solver='saga') # creamos el objeto, y una semilla de randon cero 
model.fit(X_train, y_train)

LogisticRegression(C=0.01, random_state=0, solver='saga')

In [44]:
# Salvamos el modelo entrenado
import joblib as jb

jb.dump(model, 'modelo_lr.pkl')
jb.dump(sc, 'sc_lr.pkl')

['sc_lr.pkl']

In [45]:
model = jb.load('modelo_lr.pkl')

### Predicción de los resultados en el conjunto de prueba

In [58]:
y_pred = model.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0. 0.]
 [1. 1.]
 [0. 0.]
 ...
 [0. 0.]
 [1. 0.]
 [1. 1.]]


### Matriz de confucion

In [59]:
# hacer matriz pero en training
cm = confusion_matrix(y_train, model.predict(X_train))
print(cm)
accuracy_score(y_train, model.predict(X_train))

[[2093  285]
 [ 450 1020]]


0.808991683991684

In [48]:
from sklearn.metrics import f1_score

In [60]:
# Calcula el F1-score
f1 = f1_score(y_test, y_pred, average='weighted')

print("F1-score:", f1)

F1-score: 0.8095516108873383


### Ajustar hiperparametros

In [50]:
from sklearn.model_selection import GridSearchCV

In [51]:
# Definimos los parámetros que queremos ajustar
param_grid = {
    'penalty': ['l1', 'l2'],  # Tipo de regularización
    'C': [0.01, 0.1, 1, 10, 100]  # Parámetro de regularización inversa
}

In [52]:
model1 = LogisticRegression()

In [53]:
# Configuramos la búsqueda en cuadrícula
grid_search = GridSearchCV(model1, param_grid, cv=5, scoring='accuracy')


In [54]:
# Realizamos la búsqueda en cuadrícula para encontrar la mejor combinación de hiperparámetros
grid_search.fit(X, y)

d:\0_Respaldo\0_Proyectos_2024\Diabetes alert\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


d:\0_Respaldo\0_Proyectos_2024\Diabetes alert\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\0_Respaldo\0_Proyectos_2024\Diabetes alert\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [55]:
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

Mejores hiperparámetros encontrados:
{'C': 0.01, 'penalty': 'l2'}


In [56]:
# Mostramos la precisión del modelo utilizando los mejores hiperparámetros
print("Precisión del modelo con los mejores hiperparámetros:", grid_search.best_score_)

Precisión del modelo con los mejores hiperparámetros: 0.8054890282072161
